# Query Text Data

Demonstrates use of the Intelligence Toolkit library to respond to queries about a collection of text documents.

See [readme](https://github.com/microsoft/intelligence-toolkit/blob/main/app/workflows/query_text_data/README.md) for more details.


In [1]:
import sys
sys.path.append("..")
import os
os.environ['MKL_THREADING_LAYER'] = 'GNU' # Avoids threadpoolctl error in Linux and MacOS
from toolkit.query_text_data import QueryTextData, ProcessedChunks, ChunkSearchConfig, AnswerConfig, AnswerObject
import toolkit.query_text_data.prompts as prompts
from toolkit.AI.openai_configuration import OpenAIConfiguration
from toolkit.AI.openai_embedder import OpenAIEmbedder
from toolkit.helpers.constants import CACHE_PATH
import nest_asyncio # Necessary to run async code in ipynb
import pandas as pd
nest_asyncio.apply()

c:\Users\daedge\AppData\Local\pypoetry\Cache\virtualenvs\intelligence-toolkit-lXFNld9n-py3.11\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\daedge\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\daedge\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\daedge\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [2]:
# Create the workflow object
qtd = QueryTextData()
# Set up the AI model and embedding model
ai_configuration = OpenAIConfiguration(
    {
        "api_type": "OpenAI",
        "api_key": os.environ["OPENAI_API_KEY"],
        "model": "gpt-4o",
    }
)
qtd.set_ai_config(
    ai_configuration=ai_configuration,
    embedding_cache=CACHE_PATH
)
text_embedder = OpenAIEmbedder(
    configuration=ai_configuration,
)
qtd.set_embedder(text_embedder)
print("Created QueryTextData object")

Created QueryTextData object


In [3]:
# Provide text inputs as a dictionary of title->text
# Enter the path to your own data here
input_path = "../example_outputs/query_text_data/news_articles/news_articles_texts.csv"
file_name = input_path.split("/")[-1]
df = pd.read_csv(input_path)
text_to_chunks = qtd.process_data_from_df(df, file_name)
print("Processed data from df")

Processed data from df


In [4]:
# Process the chunks into the index data structures
processed_chunks: ProcessedChunks = qtd.process_text_chunks()
print(f"Processed chunks")

Processed chunks


In [5]:
# Embed the text chunks
cid_to_vector = await qtd.embed_text_chunks()
print(f"Embedded chunks")

Got 501 existing texts
Got 0 new texts
Embedded chunks


In [9]:
# Edit the query to be answered
query = "What events are discussed?"
expanded_query = await qtd.anchor_query_to_concepts(
    query=query,
    top_concepts=100
)
print(f"Expanded query: {expanded_query}")

Expanded query: What events are discussed in the context of renowned artists, artificial intelligence, healthcare reform, tech innovators, culinary arts, international tennis, political landscape, and global relief efforts?


In [10]:
# Mine relevant chunks to the query
chunk_search_config: ChunkSearchConfig = ChunkSearchConfig(
    # How many relevance tests are permitted per query. Higher values may provide higher quality results at higher cost
    relevance_test_budget=50,
    # How many chunks before and after each relevant chunk to test, once the relevance test budget is near or the search process has terminated
    adjacent_test_steps=1, 
    # How many relevance tests to run on each community in turn
    community_relevance_tests=5,
    # How many relevance tests to run in parallel at a time
    relevance_test_batch_size=5,
    # How many chunks to use to rank communities by relevance
    community_ranking_chunks=5,
    # When to restart testing communities in relevance order
    irrelevant_community_restart=5
)
relevant_cids, search_summary = await qtd.detect_relevant_text_chunks(
    query=query,
    expanded_query=expanded_query,
    chunk_search_config=chunk_search_config
)
print(f"Mined relevant chunks")

Top semantic search cids: [213, 104, 50, 38, 449, 76, 178, 182, 29, 359, 344, 484, 376, 314, 464, 381, 399, 181, 93, 324, 179, 150, 202, 409, 144, 155, 441, 96, 279, 64, 229, 237, 56, 189, 55, 364, 370, 219, 251, 400, 388, 84, 62, 341, 284, 130, 47, 485, 277, 31, 183, 310, 196, 239, 267, 475, 290, 470, 457, 138, 440, 112, 296, 446, 164, 180, 24, 418, 224, 493, 4, 468, 168, 173, 380, 244, 188, 90, 233, 382, 65, 349, 442, 348, 14, 304, 109, 337, 105, 379, 391, 408, 166, 443, 165, 216, 72, 374, 20, 152]
Level 0 community sequence: ['1.1', '1.16', '1.8', '1.6', '1.3', '1.11', '1.5', '1.14', '1.7', '1.19', '1.4', '1.12', '1.10', '1.9', '1.18', '1.20', '1.2', '1.17', '1.13', '1.15']
Level 1 community sequence: ['1.1.1', '1.1.6', '1.1.2', '1.1.4', '1.8.1', '1.1.5', '1.16.1', '1.11.1', '1.1.3', '1.16.2', '1.19', '1.16.4', '1.14.2', '1.11.2', '1.5.1', '1.4.1', '1.11.3', '1.14.4', '1.8.2', '1.1.7', '1.3.1', '1.7.4', '1.9.3', '1.18.3', '1.12.5', '1.8.3', '1.5.3', '1.8.4', '1.6.14', '1.6.19', '1.3

100%|██████████| 5/5 [00:00<00:00,  6.12it/s]


Community 1 relevant? True
Incrementing level
New level 0 loop after 5 tests
Community sequence: ['1.1', '1.16', '1.8', '1.6', '1.3', '1.11', '1.5', '1.14', '1.7', '1.19', '1.4', '1.12', '1.10', '1.9', '1.18', '1.20', '1.2', '1.17', '1.13', '1.15']
Assessing relevance for community 1.1 with chunks [178, 182, 29, 359, 344]


100%|██████████| 5/5 [00:00<00:00,  5.77it/s]


Community 1.1 relevant? True
Assessing relevance for community 1.16 with chunks [76, 93, 261, 458, 490]


...

Community 1.10 relevant? False
Assessing relevance for community 1.9 with chunks [278, 162, 281, 358, 421]


0it [00:00, ?it/s]


Community 1.9 relevant? False
Assessing relevance for community 1.18 with chunks [374, 58, 40, 488]


0it [00:00, ?it/s]

Community 1.18 relevant? False
5 successive irrelevant communities; restarting
Incrementing level
Mined relevant chunks


In [11]:
# Generate an extended answer to the query, which could then be summarized into a shorter form
answer_config = AnswerConfig(
    extract_claims = False, # Whether to extract claims from relevant text chunks
    claim_search_depth = 0, # If extracting claims, how many chunks to search for additional claim support
    target_chunks_per_cluster = 5, # How many chunks to aim to analyze together in a single LLM call
)
await qtd.answer_query_with_relevant_chunks(
    answer_config=answer_config
)
print("Answered query")

c:\Users\daedge\AppData\Local\pypoetry\Cache\virtualenvs\intelligence-toolkit-lXFNld9n-py3.11\Lib\site-packages\threadpoolctl.py:1214: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)
100%|██████████| 10/10 [00:14<00:00,  1.42s/it]


Extracted references: [21, 29, 38, 47, 50, 65, 76, 87, 93, 99, 104, 166, 179, 182, 184, 213, 230, 252, 277, 320, 327, 344, 359, 388, 449, 490]
Answered query


In [12]:
# Output the final extended answer
print(qtd.answer_object.extended_answer)

# Report

## Query

*What events are discussed?*

## Expanded Query

*What events are discussed in the context of renowned artists, artificial intelligence, healthcare reform, tech innovators, culinary arts, international tennis, political landscape, and global relief efforts?*

## Answer

The events discussed include Springfield's Cultural Week, San Francisco's Green Earth Festival, the Spring Festival at Botanical Gardens, Sunset Cinema's Annual Film Festival, Miami Spring Festival, Barcelona Spring Festival, global conferences on climate action, a policy debate on education reform in Capital City, Senator Jane Smith's re-election campaign, Congresswoman Lisa Brown's environmental legislation, community engagement initiatives in the Bay Area, Boston Schools' Education Week, Hillside Marathon, Hope Fundraiser Gala, Annual Charity Gala, Chloe Reed's art workshop, David Lin's event at the City Library, Springfield Community Festival, Global Economic Forums, IMF's global economic discuss

In [13]:
# Condense the answer
qtd.condense_answer(ai_instructions=prompts.user_prompt)
print("Condensed answer")

Condensed answer


In [14]:
# Output the final extended answer
print(qtd.condensed_answer)

# Diverse Events Highlighting Cultural, Environmental, and Economic Themes

## Introduction

This report explores a variety of events that span cultural, environmental, and economic themes. These events, ranging from local festivals to global conferences, illustrate the interconnectedness of community engagement, sustainability efforts, and economic strategies. Each event serves a unique purpose, from fostering cultural appreciation and environmental awareness to addressing economic challenges and political reforms.

## Cultural Events: Celebrating Diversity and Community Engagement

Cultural events play a crucial role in celebrating diversity and fostering community engagement. They provide platforms for showcasing local talent and cultural heritage, thereby strengthening community bonds and promoting inclusivity.

### Springfield's Cultural Week

Springfield's Cultural Week, organized by the Springfield Cultural Committee, is a week-long celebration featuring art exhibitions, worksho